<center><img src="../logo.png" alt="Header" style="width: 800px;"/></center>


@Copyright (C): 2010-2020, Shenzhen Yahboom Tech  
@Author: Liusen  
@Date: 2020-02-04 17:20:02  
@LastEditors: Liusen  
@LastEditTime: 2020-09-30 18:50:02   

In [ ]:
import YB_Pcb_Car    #Import Yahboom library
car = YB_Pcb_Car.YB_Pcb_Car()

In [ ]:
import enum
import cv2

def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])

In [ ]:

import cv2
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
import time

import threading
import inspect
import ctypes
import numpy as np

image_widget = widgets.Image(format='jpeg', width=320, height=240)
display(image_widget)

## Close thread function

In [ ]:
def _async_raise(tid, exctype):
    """raises the exception, performs cleanup if needed"""
    tid = ctypes.c_long(tid)
    if not inspect.isclass(exctype):
        exctype = type(exctype)
    res = ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, ctypes.py_object(exctype))
    if res == 0:
        raise ValueError("invalid thread id")
    elif res != 1:
        # """if it returns a number greater than one, you're in trouble,
        # and you should call it again with exc=NULL to revert the effect"""
        ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, None)
        
def stop_thread(thread):
    _async_raise(thread.ident, SystemExit)

In [ ]:
global picture
picture = np.ones((240,320,3),dtype =np.uint8)*255
image = cv2.VideoCapture(0)

image.set(3, 320)
image.set(4, 240)
image.set(5, 30)  #set frame
image.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter.fourcc('M', 'J', 'P', 'G'))
image.set(cv2.CAP_PROP_BRIGHTNESS, 62) 
image.set(cv2.CAP_PROP_CONTRAST, 63) 
image.set(cv2.CAP_PROP_EXPOSURE, 4800) 
ret, frame = image.read()
image_widget.value = bgr8_to_jpeg(picture)

In [ ]:
global g_mode
g_mode = 0
global color_x, color_y, color_radius
color_x = color_y = color_radius = 0

global g_mode
g_mode = 0


## Create an array to store red HSV color gamut data

In [ ]:
import numpy as np
global color_lower
global color_upper


In [ ]:
color_hsv  = {"red"   : ((0, 70, 72), (7, 255, 255)),
              "green" : ((54, 109, 78), (77, 255, 255)),
              "blue"  : ((92, 100, 62), (121, 251, 255)),
              "yellow": ((26, 100, 91), (32, 255, 255))}

In [ ]:
color = "red"
color_lower = np.array([color_hsv[color][0][0],color_hsv[color][0][1],color_hsv[color][0][2]])
color_upper = np.array([color_hsv[color][1][0], color_hsv[color][1][1], color_hsv[color][1][2]])

In [ ]:
import PID

direction_pid = PID.PositionalPID(0.8, 0, 0.8)


yservo_pid = PID.PositionalPID(0.8, 0.2, 0.8)


speed_pid = PID.PositionalPID(2.1, 0, 0.8)




In [ ]:
car.Ctrl_Servo(1,90)
car.Ctrl_Servo(2,90)

## Color recognition main function

In [ ]:
def Color_Recongnize():
    global color_lower, color_upper, g_mode, first_read, while_cnt
    global color_x, target_servox,picture
    t_start = time.time()
    fps = 0
    ret, frame = image.read()
    #frame = cv2.resize(frame, (300, 300))
    frame = cv2.GaussianBlur(frame,(5,5),0) 
    first_read = 1
    while_cnt = 0
    time.sleep(1)
    while True:
        ret, frame = image.read()  
        #frame = cv2.resize(frame, (300, 300))
        #frame = cv2.GaussianBlur(frame,(3,3),0)  
        hsv = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
        mask = cv2.inRange(hsv,color_lower,color_upper)   
        mask = cv2.erode(mask,None,iterations=2)
        mask = cv2.dilate(mask,None,iterations=2)
        mask = cv2.GaussianBlur(mask,(5,5),0)
        cnts = cv2.findContours(mask.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)[-2]
        if 1:
            if len(cnts) > 0:
                cnt = max (cnts, key = cv2.contourArea)
                (color_x,color_y),color_radius = cv2.minEnclosingCircle(cnt)

                if color_radius > 5:

                    cv2.circle(frame,(int(color_x),int(color_y)),int(color_radius),(255,0,255),2)  
                    direction_pid.SystemOutput = color_x
                    direction_pid.SetStepSignal(160)
                    direction_pid.SetInertiaTime(0.01, 0.1)
                    target_valuex = int(direction_pid.SystemOutput)
                    #print("color_x %f", color_x)                         
                    #print("target_valuex %d", target_valuex)   

                    yservo_pid.SystemOutput = color_y
                    yservo_pid.SetStepSignal(150)
                    yservo_pid.SetInertiaTime(0.01, 0.1)
                    target_valuey = int(1500-yservo_pid.SystemOutput)
                    target_servoy = int((target_valuey-500)/10)                        
                    #print("target_servoy %d", target_servoy)  
                    if target_servoy > 180:
                        target_servoy = 180
                    if target_servoy < 0:
                        target_servoy = 0
                    #print(target_servoy)


                    speed_pid.SystemOutput = int(color_radius)
                    speed_pid.SetStepSignal(40)
                    speed_pid.SetInertiaTime(0.01, 0.1)               
                    speed_value = int(speed_pid.SystemOutput)
                    #print("color_radius %d", color_radius)

                    if speed_value > 250:
                        speed_value = 250
                    if speed_value < 0:
                        speed_value = 0



                    speed_value_l = speed_value - target_valuex
                    speed_value_r = speed_value + target_valuex

                    if speed_value_l > 250:
                        speed_value_l = 250
                    if speed_value_l < -250:
                        speed_value_l = 250

                    if speed_value_r > 250:
                        speed_value_r = 250
                    if speed_value_r < -250:
                        speed_value_r = 250


                    car.Control_Car(speed_value_l,speed_value_r)

                    car.Ctrl_Servo(2,target_servoy)
                
                else:
                    car.Car_Stop()
                    
            else:
                car.Car_Stop()


        fps = fps + 1
        mfps = fps / (time.time() - t_start)
        cv2.putText(frame, "FPS " + str(int(mfps)), (40,40), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,255), 3)
        #cv2.putText(frame, "1" , (160,120), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,255), 3)
        picture = frame
        

In [ ]:
def video():
    global picture

    image_widget.value = bgr8_to_jpeg(picture)

##  Start

In [ ]:
thread1 = threading.Thread(target=Color_Recongnize)
thread1.setDaemon(True)
thread1.start()

In [ ]:
try:
    while True:
        video()
except:
    print("ending")

##  Close

In [ ]:
stop_thread(thread1)

In [ ]:
image.release()              #After using, we need to release the object, otherwise it will be occupied and cannot be used in the next program.

In [ ]:
car.Car_Stop()